In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import relu
from DataLoader import DataLoader

import numpy as np

In [2]:
labels = np.load("labels.npy")

training_generator = DataLoader(labels, 128)

In [3]:
model = Sequential()
model.add(Dense(768, activation='relu', input_shape=(98304,)))
# model.add(Dense(768, activation='relu'))
model.add(Dense(768, activation='sigmoid'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

2022-02-26 17:46:28.931752: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-26 17:46:28.943814: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-26 17:46:28.944177: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-26 17:46:28.945392: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [5]:
model.fit(training_generator, epochs=1)

10457/10457 [==============================] - 888s 85ms/step - loss: 4.0507 - accuracy: 0.1632


In [6]:
import time
model.save("models/model-{}".format(int(time.time())))

2022-02-26 18:17:37.281059: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: models/model-1645917457/assets
